In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import random

from lib.utils import prepare_dataset, tensorsFromPair 
from lib.models import EncoderRNN, AttnDecoderRNN
from lib.train import train

In [2]:
# !wget https://download.pytorch.org/tutorial/data.zip

In [3]:
# !unzip data.zip

In [4]:
eng, fra, pairs = prepare_dataset('data/eng-fra.txt')
print(random.choice(pairs))

Reading lines...
['je vous soutiens .', 'i m supporting you .']


In [5]:
device = 'cpu'

In [6]:
hidden_size = 256
encoder = EncoderRNN(fra.n_words, hidden_size, device).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, eng.n_words, device, dropout_p=0.1).to(device)

In [7]:
encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=1e-3)
decoder_optimizer = torch.optim.SGD(attn_decoder.parameters(), lr=1e-3)

In [8]:
train(encoder, attn_decoder, 75000, encoder_optimizer, 
      decoder_optimizer, pairs, device, source = fra, 
      target = eng, print_every=10)

[7.889074961344401]